In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# FraudFinder - Feature Engineering (batch)

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?&download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/02_feature_engineering_batch.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/02_feature_engineering_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/02_feature_engineering_batch.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[FraudFinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the FraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

As you engineer features for model training, it's important to consider how the features are computed when making predictions with new data. For online predictions, you may have features that can be pre-computed via _batch feature engineering_. You may also features that need to be computed on-the-fly via _streaming-based feature engineering_. For these Fraudfinder labs, for computing features based on the last n _days_, you will use _batch_ feature engineering in BigQuery; for computing features based on the last n _minutes_, you will use _streaming-based_ feature engineering using Dataflow.

This notebook shows how to generate new features on bank transactions by customer and terminal over the last n days, by doing batch feature engineering in SQL with BigQuery. Then, you will create a feature store using Vertex AI Feature Store, and ingest your newly-created features from BigQuery into Vertex AI Feature Store, so that a feature store can become the single source of data for both training and model inference. 


This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)


Steps performed in this notebook:

- Build customer and terminal-related features
- Create Feature store, feature groups and features
- Create the Feature View and ingest feature values in Feature store from BigQuery table
- Read features from the feature store

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [1]:
import json

try:
    with open("config_path.json", "r") as f:
        config_path = json.load(f)
except FileNotFoundError:
    print("config_path.json not found. Please make sure the file exists.")
    ID = None

In [2]:
from vertex_ai_lab.utils import read_from_bucket, VertexConfig


config = read_from_bucket(config_path["bucket"], config_path["conf_uri"])
config = VertexConfig(**config)

PROJECT_NUM = !gcloud projects list --filter="$config.PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]

### Import libraries

In [3]:
from datetime import datetime, timedelta
from typing import Union

import pandas as pd
pd.set_option('display.max_columns', 500)

# Vertex AI and Vertex AI Feature Store
from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery

### Define constants

In [4]:
# Define the date range of transactions for feature engineering (last 10 days up until yesterday)
YESTERDAY = datetime.today() - timedelta(days=1)
YEAR_MONTH_PREFIX = YESTERDAY.strftime("%Y-%m")
DATAPROCESSING_START_DATE = (YESTERDAY - timedelta(days=10)).strftime("%Y-%m-%d")
DATAPROCESSING_END_DATE = YESTERDAY.strftime("%Y-%m-%d")

# Define Vertex AI Feature store settings.
CUSTOMERS_TABLE_NAME = f"customers_{DATAPROCESSING_END_DATE.replace('-', '')}"
CUSTOMERS_BQ_TABLE_URI = f"{config.PROJECT_ID}.tx.{CUSTOMERS_TABLE_NAME}"
TERMINALS_TABLE_NAME = f"terminals_{DATAPROCESSING_END_DATE.replace('-', '')}"                    
TERMINALS_BQ_TABLE_URI = f"{config.PROJECT_ID}.tx.{TERMINALS_TABLE_NAME}"

### Helpers

Define a set of helper functions to run BigQuery query and create features. 

In [5]:
def run_bq_query(sql: str, show=False) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
        show: A flag to show query result in a Pandas Dataframe
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    result = client_result.result()
    print(f"Finished job_id: {job_id}")
    
    if show:
        df = result.to_arrow().to_pandas()
        return df

## Feature Engineering

### Define customer and terminal-related features for batch feature engineering

In this section, you will create features, based on historical customer behaviour and historical terminal activities. This features will be batch-generated using SQL in BigQuery, where the historical data is stored.

The query below will calculate 2 sets of features: 

1. **Customer-related features**: which describes the spending behaviour of customer within 1, 7 and 15 days time windows using number of transactions and average amount spent in dollars ($)

2. **Terminal-related features** which describes the risk of a given terminal to be exposed to fraudulent transactions within 1, 7 and 15 days using average number of fraudulent transactions in dollars ($), the number of transactions and risk index. One thing to note is that you will add some delay to take into account time that would pass between the time of transaction and the result of fraud investigation or customer claim.

You will use one month of transaction data starting from the end of January and going back to compute the features.

Below is the schema you should expect to see, after doing the batch feature engineering in BigQuery:

|feature_time           |customer_id| customer batch features   |
|-----------------------|-----------|---------------------------|
|2022-01-01 17:20:15 UTC|1          |(e.g., nb_tx,  avg_tx)     |
|2022-01-02 12:08:40 UTC|2          |(e.g., nb_tx,  avg_tx)     |
|2022-01-03 17:30:48 UTC|3          |(e.g., nb_tx,  avg_tx)     |


|feature_time           |terminal_id| terminal batch features|
|-----------------------|-----------|------------------------|
|2022-01-01 17:20:15 UTC|12345      |(e.g., risk_x_days)     |
|2022-01-02 12:08:40 UTC|26789      |(e.g., risk_x_days)     |
|2022-01-03 17:30:48 UTC|101112     |(e.g., risk_x_days)     |


#### Create the query to create batch features

Date settings to be used:

In [6]:
print(f"""
DATAPROCESSING_START_DATE: {DATAPROCESSING_START_DATE}
DATAPROCESSING_END_DATE: {DATAPROCESSING_END_DATE}
""")


DATAPROCESSING_START_DATE: 2025-05-05
DATAPROCESSING_END_DATE: 2025-05-15



##### Customer feature table

Customer table SQL query string:

In [7]:
create_customer_batch_features_query = f"""
CREATE OR REPLACE TABLE `{CUSTOMERS_BQ_TABLE_URI}` AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{config.RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{config.RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  -- query to calculate CUSTOMER spending behaviour --------------------------------------------------------------------------------
  get_customer_spending_behaviour AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TX_AMOUNT,
    TX_FRAUD,
    
    # calc the number of customer tx over daily windows per customer (1, 7 and 15 days, expressed in seconds)
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_1DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_7DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_14DAY_WINDOW,
      
    # calc the customer average tx amount over daily windows per customer (1, 7 and 15 days, expressed in seconds, in dollars ($))
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW,
  FROM get_raw_table)

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_timestamp,
  CUSTOMER_ID AS customer_id,
  CAST(CUSTOMER_ID_NB_TX_1DAY_WINDOW AS INT64) AS customer_id_nb_tx_1day_window,
  CAST(CUSTOMER_ID_NB_TX_7DAY_WINDOW AS INT64) AS customer_id_nb_tx_7day_window,
  CAST(CUSTOMER_ID_NB_TX_14DAY_WINDOW AS INT64) AS customer_id_nb_tx_14day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_1day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_7day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_14day_window,
FROM
  get_customer_spending_behaviour
"""

##### Run the query 

You create the customer features table

In [8]:
run_bq_query(create_customer_batch_features_query)

Finished job_id: 9d0fb020-d58b-4691-b9da-c2fac7d6accc


##### Inspect the result 

You can query some data rows to validate the result of the query

In [9]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 10",
    show=True
)

Finished job_id: 55b097e8-3f99-419a-848b-f7a915e9785c


,feature_timestamp,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window
0,2025-05-06 01:52:13+00:00,0207598195419865,3,16,16,31.386667,27.606875,27.606875
1,2025-05-15 05:38:47+00:00,2853431768962442,6,22,40,23.731667,23.089545,23.249250
2,2025-05-04 06:17:32+00:00,5000139664464007,1,4,4,21.630000,21.360000,21.360000
3,2025-05-03 03:52:39+00:00,5420454035885127,3,12,12,96.800000,97.240000,97.240000
4,2025-05-10 21:29:00+00:00,8679143165351887,6,23,34,84.521667,81.083913,81.790882
5,2025-05-10 00:41:27+00:00,0125596948516114,4,18,28,14.212500,13.357778,13.209643
6,2025-04-30 10:21:11+00:00,2785425185695032,1,1,1,50.760000,50.760000,50.760000
7,2025-05-07 04:10:31+00:00,2785425185695032,3,13,13,57.476667,59.216154,59.216154
8,2025-04-30 12:26:57+00:00,2976230031485025,2,2,2,56.910000,56.910000,56.910000
9,2025-05-07 12:36:23+00:00,5336610502816183,2,10,13,91.525000,91.341000,91.284615


##### Terminal feature table

Terminal table SQL query string:

In [10]:
create_terminal_batch_features_query = f"""
CREATE OR REPLACE TABLE `{TERMINALS_BQ_TABLE_URI}` AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{config.RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{config.RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  # query to calculate TERMINAL spending behaviour --------------------------------------------------------------------------------
  get_variables_delay_window AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    
    # calc total amount of fraudulent tx and the total number of tx over the delay period per terminal (7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_DELAY,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_TX_DELAY,
      
    # calc total amount of fraudulent tx and the total number of tx over the delayed window per terminal (window + 7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_1_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_7_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_14_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_TX_1_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_TX_7_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_TX_14_DELAY_WINDOW,
  FROM get_raw_table),

  # query to calculate TERMINAL risk factors ---------------------------------------------------------------------------------------
  get_risk_factors AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    # calculate numerator of risk index
    NB_FRAUD_1_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_1DAY_WINDOW,
    NB_FRAUD_7_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_7DAY_WINDOW,
    NB_FRAUD_14_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_14DAY_WINDOW,
    # calculate denominator of risk index
    NB_TX_1_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_1DAY_WINDOW,
    NB_TX_7_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_7DAY_WINDOW,
    NB_TX_14_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_14DAY_WINDOW,
      FROM
    get_variables_delay_window),

  # query to calculate the TERMINAL risk index -------------------------------------------------------------------------------------
  get_risk_index AS (
    SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TERMINAL_ID_NB_TX_1DAY_WINDOW,
    TERMINAL_ID_NB_TX_7DAY_WINDOW,
    TERMINAL_ID_NB_TX_14DAY_WINDOW,
    # calculate the risk index
    (TERMINAL_ID_NB_FRAUD_1DAY_WINDOW/(TERMINAL_ID_NB_TX_1DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_1DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_7DAY_WINDOW/(TERMINAL_ID_NB_TX_7DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_7DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_14DAY_WINDOW/(TERMINAL_ID_NB_TX_14DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_14DAY_WINDOW
    FROM get_risk_factors 
  )

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_timestamp,
  TERMINAL_ID AS terminal_id,
  CAST(TERMINAL_ID_NB_TX_1DAY_WINDOW AS INT64) AS terminal_id_nb_tx_1day_window,
  CAST(TERMINAL_ID_NB_TX_7DAY_WINDOW AS INT64) AS terminal_id_nb_tx_7day_window,
  CAST(TERMINAL_ID_NB_TX_14DAY_WINDOW AS INT64) AS terminal_id_nb_tx_14day_window,
  CAST(TERMINAL_ID_RISK_1DAY_WINDOW AS FLOAT64) AS terminal_id_risk_1day_window,
  CAST(TERMINAL_ID_RISK_7DAY_WINDOW AS FLOAT64) AS terminal_id_risk_7day_window,
  CAST(TERMINAL_ID_RISK_14DAY_WINDOW AS FLOAT64) AS terminal_id_risk_14day_window,
FROM
  get_risk_index
"""

##### Run the query 

You create the customer features table

In [11]:
run_bq_query(create_terminal_batch_features_query)

Finished job_id: 1d82bd6c-9185-4c8d-9de9-fbc45a1a6575


##### Inspect the result 

You can query some data rows to validate the result of the query

In [12]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 10",
    show=True
)

Finished job_id: c0cacf6a-4525-4e18-a2d1-52403a2c13ea


,feature_timestamp,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window
0,2025-05-09 17:02:26+00:00,06436927,21,55,55,0.000000,0.018182,0.018182
1,2025-05-03 15:22:33+00:00,57494533,0,0,0,0.000000,0.000000,0.000000
2,2025-05-04 11:13:27+00:00,42000435,0,0,0,0.000000,0.000000,0.000000
3,2025-04-30 19:56:56+00:00,75187224,0,0,0,0.000000,0.000000,0.000000
4,2025-05-10 15:31:00+00:00,81361404,20,60,60,0.000000,0.000000,0.000000
5,2025-05-03 19:50:09+00:00,82173530,0,0,0,0.000000,0.000000,0.000000
6,2025-05-15 17:37:21+00:00,82173530,12,114,140,0.000000,0.000000,0.000000
7,2025-05-11 08:09:48+00:00,90416428,24,89,89,0.000000,0.011236,0.011236
8,2025-05-15 09:50:50+00:00,90914987,21,118,145,0.000000,0.016949,0.013793
9,2025-05-07 23:28:45+00:00,26961822,33,33,33,0.060606,0.060606,0.060606


### Define customer and terminal-related features for _real-time_ feature engineering

To make more accurate predictions, you can also create real-time features to inspect, for example, the most recent minutes of activity for both customers and terminals.

In order to have these features available for training and inference, we first need to make sure they are defined in a BigQuery table as placeholders to be ingested into Vertex AI Feature Store.

In the query below, you initialize two sets of features for real-time feature engineering: 

1. Customer features which describes the spending behaviour of customer within 15, 30 and 60 minutes time windows using number of transactions and average amount spent in dollars ($)

2. Terminal features which describes the risk of a given terminal to be exposed to fraudulent transactions within 15, 30 and 60 minutes using average number of fraudulent transactions in dollars ($) and the number of transactions. 

To do so, you will:

- Add one column for each real time feature
- Set 0 as default values for each of them
- Update all real-time columns with default values

Then you will create the actual values for real-time feature engineering in the next `03_feature_engineering_streaming.ipynb` notebook. 

#### Define the query to initialize the real-time features.

##### Customer feature table

Customer table SQL query string:

In [13]:
initiate_real_time_customer_features_query = f"""
ALTER TABLE `{CUSTOMERS_BQ_TABLE_URI}`
ADD COLUMN customer_id_nb_tx_15min_window INT64,
ADD COLUMN customer_id_nb_tx_30min_window INT64,
ADD COLUMN customer_id_nb_tx_60min_window INT64,
ADD COLUMN customer_id_avg_amount_15min_window FLOAT64,
ADD COLUMN customer_id_avg_amount_30min_window FLOAT64,
ADD COLUMN customer_id_avg_amount_60min_window FLOAT64;

ALTER TABLE `{CUSTOMERS_BQ_TABLE_URI}`
ALTER COLUMN customer_id_nb_tx_15min_window SET DEFAULT 0,
ALTER COLUMN customer_id_nb_tx_30min_window SET DEFAULT 0,
ALTER COLUMN customer_id_nb_tx_60min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_15min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_30min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_60min_window SET DEFAULT 0;

UPDATE `{CUSTOMERS_BQ_TABLE_URI}`
SET customer_id_nb_tx_15min_window = 0,
    customer_id_nb_tx_30min_window  = 0,
    customer_id_nb_tx_60min_window  = 0, 
    customer_id_avg_amount_15min_window = 0,
    customer_id_avg_amount_30min_window  = 0,
    customer_id_avg_amount_60min_window  = 0
WHERE TRUE; 
"""

In [14]:
initiate_real_time_terminal_features_query = f"""
ALTER TABLE `{TERMINALS_BQ_TABLE_URI}`
ADD COLUMN terminal_id_nb_tx_15min_window INT64,
ADD COLUMN terminal_id_nb_tx_30min_window INT64,
ADD COLUMN terminal_id_nb_tx_60min_window INT64,
ADD COLUMN terminal_id_avg_amount_15min_window FLOAT64,
ADD COLUMN terminal_id_avg_amount_30min_window FLOAT64,
ADD COLUMN terminal_id_avg_amount_60min_window FLOAT64;

ALTER TABLE `{TERMINALS_BQ_TABLE_URI}`
ALTER COLUMN terminal_id_nb_tx_15min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_nb_tx_30min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_nb_tx_60min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_15min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_30min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_60min_window SET DEFAULT 0;

UPDATE `{TERMINALS_BQ_TABLE_URI}`
SET terminal_id_nb_tx_15min_window = 0,
    terminal_id_nb_tx_30min_window  = 0,
    terminal_id_nb_tx_60min_window  = 0,
    terminal_id_avg_amount_15min_window = 0,
    terminal_id_avg_amount_30min_window = 0,
    terminal_id_avg_amount_60min_window  = 0
WHERE TRUE; 
"""

#### Run the query above to initialize the real-time features.

In [15]:
for query in [initiate_real_time_customer_features_query, initiate_real_time_terminal_features_query]:
    run_bq_query(query)

Finished job_id: 28d5edc1-cc16-45ab-8b44-2c3b8135e415
Finished job_id: 1f3b8c87-a0b5-4646-a4fb-7fe6ccea02b2


#### Inspect BigQuery features tables

In [16]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: 7a1a0987-d278-4d67-88f7-d5d849f90d23


,feature_timestamp,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window,customer_id_nb_tx_15min_window,customer_id_nb_tx_30min_window,customer_id_nb_tx_60min_window,customer_id_avg_amount_15min_window,customer_id_avg_amount_30min_window,customer_id_avg_amount_60min_window
0,2025-04-30 06:57:52+00:00,5272441786908869,1,1,1,65.43,65.43,65.43,0,0,0,0.0,0.0,0.0
1,2025-04-30 04:12:12+00:00,2436458327544140,1,1,1,67.94,67.94,67.94,0,0,0,0.0,0.0,0.0
2,2025-04-30 01:29:51+00:00,3223286095043809,1,1,1,87.25,87.25,87.25,0,0,0,0.0,0.0,0.0
3,2025-05-01 08:03:46+00:00,8262782715761268,1,1,1,88.84,88.84,88.84,0,0,0,0.0,0.0,0.0
4,2025-05-01 06:03:29+00:00,8478017930253156,1,1,1,49.10,49.10,49.10,0,0,0,0.0,0.0,0.0


In [17]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: 058cb763-ef9b-4fe8-a5ae-74d9e9f3c5ff


,feature_timestamp,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window,terminal_id_nb_tx_15min_window,terminal_id_nb_tx_30min_window,terminal_id_nb_tx_60min_window,terminal_id_avg_amount_15min_window,terminal_id_avg_amount_30min_window,terminal_id_avg_amount_60min_window
0,2025-05-04 20:17:55+00:00,30430351,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
1,2025-05-05 14:18:30+00:00,17803600,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
2,2025-05-02 01:55:12+00:00,75607852,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
3,2025-05-04 07:10:31+00:00,95331498,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
4,2025-05-03 15:18:00+00:00,49399432,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0


Let's look at the final schema of the features table:

In [18]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{CUSTOMERS_TABLE_NAME}'
""", show=True)

Finished job_id: a584092a-af3f-4647-a8f1-37ae3bfd8584


,column_name,data_type
0,feature_timestamp,TIMESTAMP
1,customer_id,STRING
2,customer_id_nb_tx_1day_window,INT64
3,customer_id_nb_tx_7day_window,INT64
4,customer_id_nb_tx_14day_window,INT64
5,customer_id_avg_amount_1day_window,FLOAT64
6,customer_id_avg_amount_7day_window,FLOAT64
7,customer_id_avg_amount_14day_window,FLOAT64
8,customer_id_nb_tx_15min_window,INT64
9,customer_id_nb_tx_30min_window,INT64


In [19]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{TERMINALS_TABLE_NAME}'
""", show=True)

Finished job_id: d894d1c7-1e0a-47bd-b504-bb7334240c26


,column_name,data_type
0,feature_timestamp,TIMESTAMP
1,terminal_id,STRING
2,terminal_id_nb_tx_1day_window,INT64
3,terminal_id_nb_tx_7day_window,INT64
4,terminal_id_nb_tx_14day_window,INT64
5,terminal_id_risk_1day_window,FLOAT64
6,terminal_id_risk_7day_window,FLOAT64
7,terminal_id_risk_14day_window,FLOAT64
8,terminal_id_nb_tx_15min_window,INT64
9,terminal_id_nb_tx_30min_window,INT64


## Feature store for feature management

### What is a feature store?

The features generated are great examples of features that we can store the [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore). This is because:

- The features are needed for real-time prediction
- feature values in a feature store can be used for both training and serving
- if needed, features can be shared with other use cases beyond fraud detection

Vertex AI Feature Store provides a centralized repository for organizing, storing, and serving ML features. Using a central featurestore enables an organization to efficiently share, discover, and re-use ML features at scale, which can increase the velocity of developing and deploying new ML applications.

### Why would you like to set up it?

So far you've built and stored features in BigQuery. 

Now, in order to predict fraud, you want to serve those features in real-time with millisecond scale latency. In particular, when the ML gateway receives a prediction request for a specific transaction (including customer, terminal, and transaction ids), the system needs to fetch the features related to that transaction and pass them as inputs to the model for online prediction. As you can imagine, an analytical data warehouse such as BigQuery is not able to provide low-latency near real-time read operations. 

Vertex AI Feature Store provides a managed service for low latency scalable feature serving. It also provides a centralized feature repository with easy APIs to search and discover features, as well as feature monitoring capabilities to track drift and other quality issues. 

Vertex AI Feature Store uses a time series data model to store a series of values for features, which enables Vertex AI Feature Store to maintain feature values as they change over time and to support point-in-time queries of feature values. Feature Store organizes resources hierarchically (`Featurestore -> Feature View -> Feature Group -> Feature`) in the following order: 

- **Featurestore**: the resource to contains entities and features.
    - **FeatureView**: the serving of features.
    - **Featuregroup**: a feature group is feature registry to present a set of features.
        - **Feature**: under a feature group, a feature is an attribute.


You must create these resources before you can ingest data into a Feature View. 

In the follow section, you will use create a feature store using Vertex AI Feature Store, and ingest data into it to be used later for training and model inference.

### Initialize Vertex AI SDK

Initialize the Vertex AI SDK to get access to Vertex AI services programmatically. 

In [4]:
vertex_ai.init(project=config.PROJECT_ID, location=config.REGION, staging_bucket=config.BUCKET_NAME)

In [5]:
# Create the feature group for customers
# Use google-cloud-aiplatform instead of vertexai as it has more features available
from google.api_core import operation as gac_operation
from google.cloud import aiplatform_v1
from google.cloud.aiplatform_v1 import types as vertexai_types


def create_feature_group(
    project_id: str,
    location: str,
    feature_group_id: str,
    bq_table_uri: str,
    entity_id_columns: list[str],
    ts_col: str = "feature_timestamp",
    desc: str | None = None,
) -> gac_operation.Operation:
    client = aiplatform_v1.FeatureRegistryServiceClient(
        client_options={"api_endpoint": f"{location}-aiplatform.googleapis.com"}
    )
    fg_pb = vertexai_types.FeatureGroup(
        big_query=vertexai_types.FeatureGroup.BigQuery(
            big_query_source=vertexai_types.BigQuerySource(input_uri=bq_table_uri),
            entity_id_columns=entity_id_columns,
            time_series=vertexai_types.FeatureGroup.BigQuery.TimeSeries(timestamp_column=ts_col),
        ),
        name=f"projects/{project_id}/locations/{location}/featureGroups/{feature_group_id}",
        description=desc,
    )
    req_pb = vertexai_types.feature_registry_service.CreateFeatureGroupRequest(
        parent=f"projects/{project_id}/locations/{location}",
        feature_group=fg_pb,
        feature_group_id=feature_group_id,
        
    )
    ops = client.create_feature_group(
        request=req_pb,
    )

    return ops


### Create the feature group for customer
The feature group maps to the customer feature bigquery table. Name it as `customer`.

#### Create the ```customer``` feature group

In [7]:
cfg_ops = create_feature_group(
    config.PROJECT_ID,
    config.REGION,
    config.CUSTOMER_ENTITY_ID,
    f"bq://{CUSTOMERS_BQ_TABLE_URI}",
    [config.CUSTOMER_ENTITY_ID_FIELD],
    desc="This is the customer feature group",
)
cfg_ops.result()

name: "projects/401714874268/locations/us-central1/featureGroups/customer"

### Create features in `customer` feature group

In [8]:
def create_feature(
    project_id: str,
    location: str,
    existing_feature_group_id: str,
    feature_info_ls: list[dict],
    version_column_name_ls: list[str],
):
    client = aiplatform_v1.FeatureRegistryServiceClient(
        client_options={"api_endpoint": f"{location}-aiplatform.googleapis.com"}
    )
    parent_fg_id = f"projects/{project_id}/locations/{location}/featureGroups/{existing_feature_group_id}"

    for feature_info, version_column_name in zip(feature_info_ls, version_column_name_ls):
        for feature_id, feature_id_info in feature_info.items():
            req_pb = vertexai_types.featurestore_service.CreateFeatureRequest(
                parent=parent_fg_id,
                feature=vertexai_types.Feature(
                    name=f"{parent_fg_id}/features/{feature_id}",
                    description=feature_id_info["description"],
                    labels=feature_id_info["labels"],
                    version_column_name=version_column_name,
                ),
                feature_id=feature_id,
            )
            _ = client.create_feature(
                request=req_pb,
            )
            print(f"{feature_id} - Done")

In [9]:
customer_features = [
    {
        "customer_id_nb_tx_1day_window": {
            "description": "Number of transactions by the customer in the last day",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_nb_tx_7day_window": {
            "description": "Number of transactions by the customer in the last 7 days",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_nb_tx_14day_window": {
            "description": "Number of transactions by the customer in the last 14 days",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_avg_amount_1day_window": {
            "description": "Average spending amount in the last day",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_avg_amount_7day_window": {
            "description": "Average spending amount in the last 7 days",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_avg_amount_14day_window": {
            "description": "Average spending amount in the last 14 days",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_nb_tx_15min_window": {
            "description": "Number of transactions by the customer in the last 15 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_nb_tx_30min_window": {
            "description": "Number of transactions by the customer in the last 30 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_nb_tx_60min_window": {
            "description": "Number of transactions by the customer in the last 60 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_avg_amount_15min_window": {
            "description": "Average spending amount in the last 15 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_avg_amount_30min_window": {
            "description": "Average spending amount in the last 30 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "customer_id_avg_amount_60min_window": {
            "description": "Average spending amount in the last 60 minutes",
            "labels": {"status": "passed"},
        }
    },
]
customer_version_col_names = [list(feature)[0] for feature in customer_features]

In [10]:
create_feature(
    config.PROJECT_ID,
    config.REGION,
    config.CUSTOMER_ENTITY_ID,
    customer_features,
    customer_version_col_names,
)

customer_id_nb_tx_1day_window - Done
customer_id_nb_tx_7day_window - Done
customer_id_nb_tx_14day_window - Done
customer_id_avg_amount_1day_window - Done
customer_id_avg_amount_7day_window - Done
customer_id_avg_amount_14day_window - Done
customer_id_nb_tx_15min_window - Done
customer_id_nb_tx_30min_window - Done
customer_id_nb_tx_60min_window - Done
customer_id_avg_amount_15min_window - Done
customer_id_avg_amount_30min_window - Done
customer_id_avg_amount_60min_window - Done


### Create features in `terminal` feature group

In [12]:
cfg_ops = create_feature_group(
    config.PROJECT_ID,
    config.REGION,
    config.TERMINAL_ENTITY_ID,
    f"bq://{TERMINALS_BQ_TABLE_URI}",
    [config.TERMINALS_ENTITY_ID_FIELD],
    desc="This is the terminal feature group",
)
cfg_ops.result()

name: "projects/401714874268/locations/us-central1/featureGroups/terminal"

#### Create features of the ```terminal``` entity type

In [13]:
terminal_features = [
    {
        "terminal_id_nb_tx_1day_window": {
            "description": "Number of transactions by the terminal in the last day",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_nb_tx_7day_window": {
            "description": "Number of transactions by the terminal in the 7 days",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_nb_tx_14day_window": {
            "description": "Number of transactions by the terminal in the 14 days",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_risk_1day_window": {
            "description": "Risk score calculated average number of frauds on the terminal in the last day",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_risk_7day_window": {
            "description": "Risk score calculated average number of frauds on the terminal in the last 7 days",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_risk_14day_window": {
            "description": "Risk score calculated average number of frauds on the terminal in the last 14 day",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_nb_tx_15min_window": {
            "description": "Number of transactions by the terminal in the last 15 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_nb_tx_30min_window": {
            "description": "Number of transactions by the terminal in the last 30 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_nb_tx_60min_window": {
            "description": "Number of transactions by the terminal in the last 60 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_avg_amount_15min_window": {
            "description": "Average spending amount in the last 15 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_avg_amount_30min_window": {
            "description": "Average spending amount in the last 30 minutes",
            "labels": {"status": "passed"},
        }
    },
    {
        "terminal_id_avg_amount_60min_window": {
            "description": "Average spending amount in the last 60 minutes",
            "labels": {"status": "passed"},
        }
    },
]
terminal_version_col_names = [list(feature)[0] for feature in terminal_features]

In [14]:
create_feature(
    config.PROJECT_ID,
    config.REGION,
    config.TERMINAL_ENTITY_ID,
    terminal_features,
    terminal_version_col_names
)

terminal_id_nb_tx_1day_window - Done
terminal_id_nb_tx_7day_window - Done
terminal_id_nb_tx_14day_window - Done
terminal_id_risk_1day_window - Done
terminal_id_risk_7day_window - Done
terminal_id_risk_14day_window - Done
terminal_id_nb_tx_15min_window - Done
terminal_id_nb_tx_30min_window - Done
terminal_id_nb_tx_60min_window - Done
terminal_id_avg_amount_15min_window - Done
terminal_id_avg_amount_30min_window - Done
terminal_id_avg_amount_60min_window - Done


### Inspect your feature store in the Vertex AI console

You can also inspect your feature store in the [Vertex AI Feature Store console](https://console.cloud.google.com/vertex-ai/features)

### Ingest feature values in Vertex AI Online Feature Store

Now we need to ingest the actual feature values you created in BigQuery into the Vertex AI Feature Store.

To ingest features values in Vertex AI Feature Store, you need to create an online feature store instance, either for Bigtable serving or optimized serving.

In the following queries, you create a bigtable online store.

In [15]:
def create_bigtable_feature_online_store(
    project_id: str,
    location: str,
    feature_online_store_id: str,
    min_node_count: int,
    max_node_count: int,

) -> gac_operation.Operation:
    client = aiplatform_v1.FeatureOnlineStoreAdminServiceClient(
        client_options={"api_endpoint": f"{location}-aiplatform.googleapis.com"}
    )
    req_pb = vertexai_types.feature_online_store_admin_service.CreateFeatureOnlineStoreRequest(
        parent=f"projects/{project_id}/locations/{location}",
        feature_online_store=vertexai_types.FeatureOnlineStore(
            bigtable=vertexai_types.FeatureOnlineStore.Bigtable(
                auto_scaling=vertexai_types.FeatureOnlineStore.Bigtable.AutoScaling(
                    min_node_count=min_node_count,
                    max_node_count=max_node_count,
                )
            ),
            name=feature_online_store_id,
        ),
        feature_online_store_id=feature_online_store_id,
    )

    ops = client.create_feature_online_store(request=req_pb)

    return ops


In [ ]:
cbfos_ops = create_bigtable_feature_online_store(
    config.PROJECT_ID,
    config.REGION,
    config.FEATURESTORE_ID,
    config.ONLINE_STORAGE_NODES,
    config.ONLINE_STORAGE_NODES
)
cbfos_ops.result()

### Create feature view
A feature view is required to bind to the online store in order to serve features. During the feature view creation, you can associate features either from the feature group, or from bigquery data source.

Below we create the view with direct feature group association.

In [17]:
def create_feature_view_from_feature_registry(
    project_id: str,
    location: str,
    feature_online_store_id: str,
    feature_view_id: str,
    feature_groups: dict[str, list[str]],
    
) -> gac_operation.Operation:
    client = aiplatform_v1.FeatureOnlineStoreAdminServiceClient(
        client_options={"api_endpoint": f"{location}-aiplatform.googleapis.com"}
    )
    req_pb = vertexai_types.feature_online_store_admin_service.CreateFeatureViewRequest(
        parent=f"projects/{project_id}/locations/{location}/featureOnlineStores/{feature_online_store_id}",
        feature_view = vertexai_types.FeatureView(
            feature_registry_source=vertexai_types.FeatureView.FeatureRegistrySource(
                feature_groups=[
                    vertexai_types.FeatureView.FeatureRegistrySource.FeatureGroup(
                        feature_group_id=feature_group_id,
                        feature_ids=feature_ids,
                    ) for feature_group_id, feature_ids in feature_groups.items()
                ]
            ),
            sync_config=vertexai_types.FeatureView.SyncConfig(
                cron="0 0 * * 6",
            )
        ),
        feature_view_id=feature_view_id,
        run_sync_immediately=True,
    )

    ops = client.create_feature_view(
        request=req_pb
    )

    return ops


In [ ]:
feature_groups = {
    config.CUSTOMER_ENTITY_ID: customer_version_col_names,
    config.TERMINAL_ENTITY_ID: terminal_version_col_names,
}
cfvffr_ops = create_feature_view_from_feature_registry(
    config.PROJECT_ID,
    config.REGION,
    config.FEATURESTORE_ID,
    config.FEATUREVIEW_ID,
    feature_groups,
)
cfvffr_ops.result()

This section takes awhile

### Check feature store values from the view

In [38]:
from google.protobuf.json_format import MessageToDict


client = aiplatform_v1.FeatureOnlineStoreServiceClient(client_options={"api_endpoint": f"{config.REGION}-aiplatform.googleapis.com"})
fv_req_pb = vertexai_types.feature_online_store_service.FetchFeatureValuesRequest(
    feature_view=f"projects/{PROJECT_NUM}/locations/{config.REGION}/featureOnlineStores/{config.FEATURESTORE_ID}/featureViews/{config.FEATUREVIEW_ID}",
    data_key=vertexai_types.feature_online_store_service.FeatureViewDataKey(
        key="5830444124423549",
    ),
    data_format=vertexai_types.feature_online_store_service.FeatureViewDataFormat.PROTO_STRUCT,
)
fv_resp = client.fetch_feature_values(request=fv_req_pb)

In [39]:
MessageToDict(fv_resp._pb)

{'protoStruct': {'customer_id_nb_tx_7day_window': 10.0,
  'customer_id_avg_amount_1day_window': 56.11,
  'terminal_id_nb_tx_15min_window': None,
  'customer_id_nb_tx_1day_window': 1.0,
  'terminal_id_nb_tx_1day_window': None,
  'customer_id_nb_tx_60min_window': 0.0,
  'terminal_id_nb_tx_30min_window': None,
  'customer_id_avg_amount_7day_window': 53.859,
  'terminal_id_nb_tx_14day_window': None,
  'customer_id_nb_tx_30min_window': 0.0,
  'terminal_id_risk_7day_window': None,
  'customer_id_avg_amount_15min_window': 0.0,
  'customer_id_avg_amount_60min_window': 0.0,
  'terminal_id_avg_amount_15min_window': None,
  'terminal_id_risk_14day_window': None,
  'customer_id_avg_amount_30min_window': 0.0,
  'terminal_id_avg_amount_30min_window': None,
  'customer_id_nb_tx_15min_window': 0.0,
  'terminal_id_avg_amount_60min_window': None,
  'feature_timestamp': 1747362100769273.0,
  'customer_id_avg_amount_14day_window': 54.514444444,
  'terminal_id_nb_tx_7day_window': None,
  'terminal_id_nb_tx

### END

Now you can go to the next notebook `03_feature_engineering_streaming.ipynb` within vertex_ai directory

## Clean up

In [ ]:
# ff_feature_store.delete(sync=True, force=True)